### Comparing Models and Vectorization Strategies for Text Classification

This try-it focuses on weighing the positives and negatives of different estimators and vectorization strategies for a text classification problem.  In order to consider each of these components, you should make use of the `Pipeline` and `GridSearchCV` objects in scikitlearn to try different combinations of vectorizers with different estimators.  For each of these, you also want to use the `.cv_results_` to examine the time for the estimator to fit the data.

### The Data

The dataset below is from [kaggle]() and contains a dataset named the "ColBert Dataset" created for this [paper](https://arxiv.org/pdf/2004.12765.pdf).  You are to use the text column to classify whether or not the text was humorous.  It is loaded and displayed below.

**Note:** The original dataset contains 200K rows of data. It is best to try to use the full dtaset. If the original dataset is too large for your computer, please use the 'dataset-minimal.csv', which has been reduced to 100K.

In [1]:
import pandas as pd

In [17]:
df = pd.read_csv('try-it_18_1_starter/text_data/dataset.csv')

In [18]:
df.head()

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    200000 non-null  object
 1   humor   200000 non-null  bool  
dtypes: bool(1), object(1)
memory usage: 1.7+ MB


#### Task


**Text preprocessing:** As a pre-processing step, perform both `stemming` and `lemmatizing` to normalize your text before classifying. For each technique use both the `CountVectorize`r and `TfidifVectorizer` and use options for stop words and max features to prepare the text data for your estimator.

**Classification:** Once you have prepared the text data with stemming lemmatizing techniques, consider `LogisticRegression`, `DecisionTreeClassifier`, and `MultinomialNB` as classification algorithms for the data. Compare their performance in terms of accuracy and speed.

Share the results of your best classifier in the form of a table with the best version of each estimator, a dictionary of the best parameters and the best score.

In [5]:
pd.DataFrame({'model': ['Logistic', 'Decision Tree', 'Bayes'], 
             'best_params': ['', '', ''],
             'best_score': ['', '', '']}).set_index('model')

,best_params,best_score
model,,
Logistic,,
Decision Tree,,
Bayes,,


In [16]:
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/kellen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kellen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/kellen/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Stemming

In [21]:
def stemmer(text):
    lst = word_tokenize(text)
    stem = PorterStemmer()
    return ' '.join([stem.stem(i) for i in lst])


In [15]:
print(stemmer("Joe biden rules out 2020 bid: 'guys, i'm not"))

joebidenruleout2020bid:'guy,i'mnot


In [22]:
df['stemmer'] = df['text'].apply(stemmer)
df.head()

,text,humor,stemmer
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False,"joe biden rule out 2020 bid : 'guy , i 'm not ..."
1,Watch: darvish gave hitter whiplash with slow ...,False,watch : darvish gave hitter whiplash with slow...
2,What do you call a turtle without its shell? d...,True,what do you call a turtl without it shell ? de...
3,5 reasons the 2016 election feels so personal,False,5 reason the 2016 elect feel so person
4,"Pasco police shot mexican migrant from behind,...",False,"pasco polic shot mexican migrant from behind ,..."


### Lemmatizing

In [25]:
def lemma(text):
    lst = word_tokenize(text)
    lemma = WordNetLemmatizer()
    return ' '.join([lemma.lemmatize(i) for i in lst])
    

In [26]:
print(lemma("Joe biden rules out 2020 bid: 'guys, i'm not"))

Joe biden rule out 2020 bid : 'guys , i 'm not


In [27]:
df['lemma'] = df['text'].apply(lemma)
df.head()

,text,humor,stemmer,lemma
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False,"joe biden rule out 2020 bid : 'guy , i 'm not ...","Joe biden rule out 2020 bid : 'guys , i 'm not..."
1,Watch: darvish gave hitter whiplash with slow ...,False,watch : darvish gave hitter whiplash with slow...,Watch : darvish gave hitter whiplash with slow...
2,What do you call a turtle without its shell? d...,True,what do you call a turtl without it shell ? de...,What do you call a turtle without it shell ? d...
3,5 reasons the 2016 election feels so personal,False,5 reason the 2016 elect feel so person,5 reason the 2016 election feel so personal
4,"Pasco police shot mexican migrant from behind,...",False,"pasco polic shot mexican migrant from behind ,...",Pasco police shot mexican migrant from behind ...


### Count Vectorization

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [29]:
X1 = df[['stemmer']]
y1 = df['humor']

In [30]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,random_state = 42)
X1_train.head()

,stemmer
21743,9 fact you probabl would n't believ 50 year ago
124554,knock knock ? ? who 's there ? ? jehovah wit ....
10351,i 'm not an expert on masturb but i hold my own
135164,dad demand apolog from ann coulter for use 're...
49969,whi us it not good to have an oili face ? the ...


In [31]:
X1_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150000 entries, 21743 to 121958
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   stemmer  150000 non-null  object
dtypes: object(1)
memory usage: 2.3+ MB
